In [13]:
import pandas as pd
import altair as alt

df = pd.read_csv("Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv")
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1947464.0,1948627.0,1950816.0,1948261.0,1963061.0,1991246.0,2032631.0,2061282.0,2073202.0,2085824.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,378936.0,...,703470.0,707917.0,713071.0,717189.0,720446.0,721439.0,725713.0,728803.0,736222.0,742394.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,2864577.0,2866022.0,2864102.0,2855516.0,2862423.0,2865414.0,2895946.0,2922250.0,2971024.0,2996826.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,225068.0,...,421730.0,429708.0,436807.0,442558.0,448665.0,458218.0,469926.0,483728.0,495207.0,502760.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,497125.0,...,942602.0,947407.0,954916.0,960563.0,966117.0,967202.0,972661.0,976754.0,986001.0,994428.0


In [18]:
df.rename(columns = {'RegionName':'ZipCode'}, inplace = True)

In [19]:
df_melt = df.melt(id_vars=['RegionID','SizeRank','ZipCode','RegionType','StateName','State','City','Metro','CountyName'], var_name='Date', value_name='Price')

In [24]:
df_zip = df_melt.loc[df_melt['ZipCode'].isin([20723, 21030, 21084, 21701, 21074, 21222])]

In [25]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')

# The basic line
line = alt.Chart().mark_line(interpolate='basis').encode(
    alt.X('Date:T', axis=alt.Axis(title='')),
    alt.Y('Price:Q', axis=None),
    #alt.Y('price:Q', axis=alt.Axis(title='',format='$f')),
    #color='RegionName:N'
    color=alt.Color('ZipCode:N', legend=alt.Legend(title="Zip Code")),
    tooltip=['Price:Q','ZipCode:N','CountyName:O','Date:T'],
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart().mark_point().encode(
    x='Date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Price:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart().mark_rule(color='gray').encode(
    x='Date:T',
).transform_filter(
    nearest
)

chart1 = alt.layer(line, selectors, points, rules, text,
                       data=df_zip, 
                       width=600, height=300,title='Maryland Home Value History (Select Zip Codes)')

chart2 = alt.Chart(df3).mark_boxplot(extent='min-max').encode(
    x=alt.X('ZipCode:O', axis=alt.Axis(title=None)),
    y='Price:Q',
)

chart2 | chart1

#comp_chart = (chart2 | chart1)
#comp_chart.save('md-zip-hvi.html')

alt.HConcatChart(...)